# Quantum Transpilation Analysis with SupermarQ

## Interactive Tutorial Notebook

Welcome to this interactive tutorial on quantum circuit transpilation analysis! This notebook provides hands-on experience with:

- **Quantum Algorithm Implementation** - Six fundamental quantum algorithms
- **Circuit Transpilation** - Optimizing circuits for real quantum hardware
- **Performance Benchmarking** - Using SupermarQ metrics to evaluate quality
- **Comparative Analysis** - Understanding trade-offs across optimization levels

### Learning Objectives

By the end of this tutorial, you will:
1. Understand how quantum transpilation affects circuit performance
2. Be able to analyze quantum algorithms using SupermarQ metrics
3. Visualize and interpret optimization trade-offs
4. Implement custom quantum benchmarking experiments

### Prerequisites

- Basic understanding of quantum computing concepts (qubits, gates, superposition)
- Familiarity with Python programming
- Qiskit and SupermarQ installed (see setup section below)

---

## Section 1: Environment Setup and Verification

Let's start by importing required libraries and verifying the installation.

In [ ]:
from qiskit.providers.fake_provider import FakeManilaV2# FakeManilaV2 is a 5-qubit backend with realistic noise characteristics# Compatible with Qiskit 1.0+

### Backend Configuration

We'll use Qiskit's `Fake7QPulseV1` backend, which simulates a 7-qubit IBM quantum device with:
- **Linear connectivity** - Qubits arranged in a line
- **Realistic noise model** - Based on real hardware characteristics
- **Gate timing** - Simulates actual gate execution times

This backend helps us understand how circuits will perform on real quantum hardware.

In [ ]:
# Initialize the backendbackend = FakeManilaV2()print(f"Backend: {backend.name}")print(f"Number of qubits: {backend.configuration().n_qubits}")print(f"\nCoupling map (qubit connectivity):")print(backend.configuration().coupling_map)# Visualize the backend topologytry:    plot_gate_map(backend)    plt.title("Backend Qubit Topology")    plt.show()except:    print("Note: Graphical visualization not available in this environment")

---
## Section 2: Understanding Quantum Transpilation

### What is Transpilation?

**Transpilation** is the process of converting an abstract quantum circuit into one that can run on specific quantum hardware. This involves:

1. **Basis Gate Translation** - Converting gates to the native gate set
2. **Qubit Mapping** - Assigning logical qubits to physical qubits
3. **SWAP Insertion** - Adding SWAP gates for qubit connectivity
4. **Circuit Optimization** - Reducing gate count and depth

### Optimization Levels

Qiskit provides four optimization levels:

- **Level 0**: Minimal optimization - basic translation only
- **Level 1**: Light optimization - basic gate commutation and cancellation
- **Level 2**: Medium optimization - more aggressive gate optimization
- **Level 3**: Heavy optimization - extensive circuit optimization

Let's see this in action with a simple example:

In [ ]:
# Create a simple 3-qubit circuit
demo_circuit = QuantumCircuit(3)
demo_circuit.h(0)
demo_circuit.cx(0, 1)
demo_circuit.cx(1, 2)
demo_circuit.h(2)
demo_circuit.measure_all()

print("Original Circuit:")
print(demo_circuit)
print(f"Depth: {demo_circuit.depth()}, Gates: {demo_circuit.size()}")

# Transpile at different levels
print("\n" + "="*50)
for level in range(4):
    transpiled = transpile(demo_circuit, backend=backend, optimization_level=level)
    print(f"\nLevel {level} - Depth: {transpiled.depth()}, "
          f"Gates: {transpiled.size()}, "
          f"CX count: {transpiled.count_ops().get('cx', 0)}")

**Observation**: Notice how higher optimization levels can reduce circuit depth and gate count!

---
## Section 3: Quantum Algorithms - Deep Dive

Now let's implement and analyze six fundamental quantum algorithms.

### Algorithm 1: Grover's Search

**Purpose**: Unstructured database search with quadratic speedup

**Complexity**: O(√N) vs O(N) classical

**How it works**:
1. Initialize qubits in superposition
2. Apply oracle to mark target state
3. Apply diffusion operator to amplify marked state
4. Repeat for optimal number of iterations

In [ ]:
def create_grover_circuit():
    """Create a 7-qubit Grover's Search circuit."""
    # Oracle: marks state |1111110⟩
    oracle = QuantumCircuit(7)
    oracle.mcx(list(range(6)), 6)
    
    # Create Grover operator
    grover_op = GroverOperator(oracle)
    
    # Build circuit
    circuit = QuantumCircuit(7)
    circuit.h(range(7))  # Superposition
    circuit.compose(grover_op, inplace=True)
    circuit.compose(grover_op, inplace=True)  # 2 iterations
    circuit.measure_all()
    circuit.name = "Grover's Search"
    return circuit

# Create and visualize
grover = create_grover_circuit()
print(f"Grover Circuit - Depth: {grover.depth()}, Qubits: {grover.num_qubits}")
print(f"Gate count: {grover.size()}")

# Draw circuit (decomposed to see structure)
print("\nCircuit diagram (first 5 levels):")
grover.draw('mpl', fold=-1)
plt.show()

### Algorithm 2: Hamiltonian Simulation

**Purpose**: Simulate quantum system dynamics

**Applications**: Quantum chemistry, materials science

**Key Concept**: Implements time evolution U = e^(-iHt) for Hamiltonian H

In [ ]:
def create_hamiltonian_sim_circuit():
    """Create a 7-qubit Hamiltonian Simulation circuit."""
    # Hamiltonian: Z ⊗ I ⊗ I^⊗5
    pauli_z = np.array([[1, 0], [0, -1]])
    identity = np.eye(2)
    identity_32 = np.eye(2**5)
    
    hamiltonian = Operator(np.kron(np.kron(pauli_z, identity), identity_32))
    
    time = 1.0
    circuit = QuantumCircuit(7)
    circuit.append(HamiltonianGate(hamiltonian, time), range(7))
    circuit.measure_all()
    circuit.name = "Hamiltonian Sim"
    return circuit

hamiltonian = create_hamiltonian_sim_circuit()
print(f"Hamiltonian Simulation - Depth: {hamiltonian.depth()}")

### 🎯 Exercise 1: Create Your Own Algorithm

Try modifying the Grover circuit to search for a different target state!

In [ ]:
# Exercise: Modify the oracle to mark a different state
def create_custom_grover(target_qubits):
    """
    Create a custom Grover circuit.
    
    Args:
        target_qubits: List of qubit indices to mark
    """
    oracle = QuantumCircuit(7)
    # TODO: Add your oracle implementation here
    # Hint: Use oracle.mcx(control_qubits, target_qubit)
    
    grover_op = GroverOperator(oracle)
    circuit = QuantumCircuit(7)
    circuit.h(range(7))
    circuit.compose(grover_op, inplace=True)
    circuit.measure_all()
    return circuit

# Test your implementation
# custom_circuit = create_custom_grover([0, 1, 2])
# print(f"Custom circuit depth: {custom_circuit.depth()}")

---
## Section 4: Transpilation Analysis - Hands-On

Let's analyze how transpilation affects a single algorithm across all optimization levels.

In [ ]:
def analyze_transpilation(circuit, backend, algorithm_name):
    """Analyze transpilation at all optimization levels."""
    print(f"\nAnalyzing: {algorithm_name}")
    print("="*60)
    
    results = {}
    
    for level in range(4):
        # Transpile
        transpiled = transpile(
            circuit,
            backend=backend,
            optimization_level=level,
            seed_transpiler=42
        )
        
        # Extract metrics
        ops = transpiled.count_ops()
        cx_count = ops.get('cx', 0)
        depth = transpiled.depth()
        gate_count = transpiled.size()
        
        results[level] = {
            'depth': depth,
            'gates': gate_count,
            'cx_gates': cx_count
        }
        
        print(f"Level {level}: Depth={depth:3d}, Gates={gate_count:3d}, CX={cx_count:3d}")
    
    return results

# Analyze Grover's algorithm
grover_analysis = analyze_transpilation(grover, backend, "Grover's Search")

### Visualize Transpilation Impact

In [ ]:
def plot_transpilation_comparison(results, title):
    """Plot transpilation metrics comparison."""
    levels = list(results.keys())
    depths = [results[l]['depth'] for l in levels]
    cx_gates = [results[l]['cx_gates'] for l in levels]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Circuit depth
    ax1.bar(levels, depths, color='steelblue', alpha=0.7)
    ax1.set_xlabel('Optimization Level')
    ax1.set_ylabel('Circuit Depth')
    ax1.set_title('Circuit Depth by Optimization Level')
    ax1.grid(axis='y', alpha=0.3)
    
    # CX gate count
    ax2.bar(levels, cx_gates, color='coral', alpha=0.7)
    ax2.set_xlabel('Optimization Level')
    ax2.set_ylabel('Two-Qubit Gate Count')
    ax2.set_title('CX Gates by Optimization Level')
    ax2.grid(axis='y', alpha=0.3)
    
    plt.suptitle(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

plot_transpilation_comparison(grover_analysis, "Grover's Algorithm Transpilation Analysis")

---
## Section 5: SupermarQ Benchmarking

### Understanding SupermarQ Metrics

SupermarQ provides five key metrics for circuit quality:

1. **Program Communication (PC)**: Measures inter-qubit interaction requirements
2. **Critical Depth (CD)**: Longest path of dependent operations
3. **Entanglement Ratio (ER)**: Proportion of entangling operations
4. **Liveness (LV)**: Qubit utilization efficiency
5. **Parallelism (PL)**: Potential for concurrent gate execution

In [ ]:
def compute_supermarq_metrics(circuit, backend, opt_level):
    """Compute SupermarQ metrics for a circuit."""
    # Transpile first
    transpiled = transpile(
        circuit,
        backend=backend,
        optimization_level=opt_level,
        seed_transpiler=42
    )
    
    # Get SupermarQ feature vector
    features = sm.benchmark.get_application_feature_vector(transpiled)
    
    metrics = {
        'PC': features[0],  # Program Communication
        'CD': features[1],  # Critical Depth
        'ER': features[2],  # Entanglement Ratio
        'LV': features[3],  # Liveness
        'PL': features[4]   # Parallelism
    }
    
    return metrics

# Compute metrics for Grover at level 2
metrics_l2 = compute_supermarq_metrics(grover, backend, opt_level=2)

print("SupermarQ Metrics (Grover, Optimization Level 2):")
print("="*50)
for name, value in metrics_l2.items():
    print(f"{name}: {value:.4f}")

### Compare Metrics Across Levels

In [ ]:
# Compute metrics for all levels
all_metrics = {}
for level in range(4):
    all_metrics[level] = compute_supermarq_metrics(grover, backend, level)
    print(f"Level {level}: PC={all_metrics[level]['PC']:.3f}, "
          f"CD={all_metrics[level]['CD']:.3f}, "
          f"ER={all_metrics[level]['ER']:.3f}")

### 🎯 Exercise 2: Benchmark Your Circuit

Use the SupermarQ metrics to analyze your custom circuit!

In [ ]:
# Exercise: Compute SupermarQ metrics for your custom circuit
# TODO: Replace 'your_circuit' with your custom circuit

# your_metrics = compute_supermarq_metrics(your_circuit, backend, opt_level=2)
# print("Your circuit metrics:", your_metrics)

---
## Section 6: Complete Experiment - All Algorithms

Now let's run the complete analysis pipeline on all six quantum algorithms!

In [ ]:
# Import all circuit creation functions from main.py
# Or define them inline for the notebook

def create_hidden_shift_circuit():
    s = '1010101'
    n = 7
    g_circuit = QuantumCircuit(n)
    for i in range(n // 2):
        g_circuit.cx(i, i + n // 2)
    s_circuit = QuantumCircuit(n)
    for i, char in enumerate(s):
        if char == '1':
            s_circuit.x(i)
    f_circuit = s_circuit.inverse().compose(g_circuit).compose(s_circuit)
    circuit = QuantumCircuit(n, n)
    circuit.h(range(n))
    circuit.barrier()
    circuit.compose(f_circuit, inplace=True)
    circuit.barrier()
    circuit.h(range(n))
    circuit.measure(range(n), range(n))
    circuit.name = "Hidden Shift"
    return circuit

def create_amplitude_estimation_circuit():
    num_state_qubits = 5
    num_ancilla_qubits = 2
    problem_circuit = QuantumCircuit(num_state_qubits + 1)
    problem_circuit.ry(2 * np.arcsin(np.sqrt(0.5)), num_state_qubits)
    qft_circuit = QFT(num_ancilla_qubits)
    circuit = PhaseEstimation(num_ancilla_qubits, problem_circuit, iqft=qft_circuit.inverse())
    circuit.name = "Amplitude Est"
    return circuit

def create_monte_carlo_circuit():
    circuit = EfficientSU2(7, reps=2, entanglement='linear')
    circuit.measure_all()
    circuit.name = "Monte Carlo"
    return circuit

def create_shor_circuit():
    n_count = 3
    U = QuantumCircuit(4)
    U.swap(0, 1)
    U.swap(1, 2)
    U.swap(2, 3)
    c_U = U.to_gate().control()
    circuit = QuantumCircuit(n_count + 4, n_count)
    circuit.h(range(n_count))
    circuit.x(n_count)
    for q in range(n_count):
        circuit.append(c_U.power(2**q), [q] + list(range(n_count, n_count+4)))
    qft_dagger = QFT(n_count, do_swaps=False).inverse().to_gate()
    circuit.append(qft_dagger, range(n_count))
    circuit.measure(range(n_count), range(n_count))
    circuit.name = "Shor's Algorithm"
    return circuit

print("✓ All algorithm generators defined")

In [ ]:
# Run complete experiment
algorithms = {
    "Grover's Search": create_grover_circuit,
    "Hamiltonian Sim": create_hamiltonian_sim_circuit,
    "Hidden Shift": create_hidden_shift_circuit,
    "Amplitude Est": create_amplitude_estimation_circuit,
    "Monte Carlo": create_monte_carlo_circuit,
    "Shor's Algorithm": create_shor_circuit,
}

results = defaultdict(lambda: defaultdict(dict))

print("Running complete experiment...\n")

for name, circuit_func in algorithms.items():
    print(f"Processing: {name}")
    circuit = circuit_func()
    
    for level in range(4):
        transpiled = transpile(
            circuit,
            backend=backend,
            optimization_level=level,
            seed_transpiler=42
        )
        
        # Two-qubit gates
        cx_count = transpiled.count_ops().get('cx', 0)
        results[name]['two_qubit_count'][level] = cx_count
        
        # SupermarQ metrics
        features = sm.benchmark.get_application_feature_vector(transpiled)
        results[name]['PC'][level] = features[0]
        results[name]['CD'][level] = features[1]
        results[name]['ER'][level] = features[2]
        results[name]['LV'][level] = features[3]
        results[name]['PL'][level] = features[4]
    
    print(f"  ✓ Completed")

print("\n✅ Experiment complete!")

### Visualize Complete Results

In [ ]:
# Two-qubit gate counts visualization
fig, ax = plt.subplots(figsize=(12, 6))

labels = list(results.keys())
x = np.arange(len(labels))
width = 0.2

colors = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c']
for i, level in enumerate([0, 1, 2, 3]):
    counts = [results[algo]['two_qubit_count'][level] for algo in labels]
    offset = width * (i - 1.5)
    ax.bar(x + offset, counts, width, label=f'Level {level}', 
           color=colors[i], alpha=0.8)

ax.set_ylabel('Two-Qubit Gate Count', fontsize=12, fontweight='bold')
ax.set_title('Two-Qubit Gate Counts Across All Algorithms', 
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([l.replace(' ', '\n') for l in labels])
ax.legend(title='Optimization Level')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# SupermarQ metrics visualization
metrics_to_plot = ['PC', 'CD', 'ER']
metric_names = ['Program Communication', 'Critical Depth', 'Entanglement Ratio']

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (metric, name) in enumerate(zip(metrics_to_plot, metric_names)):
    ax = axes[idx]
    
    for algo in results.keys():
        levels = sorted(results[algo][metric].keys())
        values = [results[algo][metric][l] for l in levels]
        ax.plot(levels, values, marker='o', label=algo, linewidth=2)
    
    ax.set_title(name, fontweight='bold')
    ax.set_xlabel('Optimization Level')
    ax.set_ylabel(metric)
    ax.set_xticks([0, 1, 2, 3])
    ax.grid(True, alpha=0.3)
    if idx == 2:
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

---
## Section 7: Analysis and Interpretation

### Key Observations

1. **Optimization Trade-offs**:
   - Higher optimization levels generally reduce gate counts
   - But may increase transpilation time
   - Not always beneficial for all algorithms

2. **Algorithm Characteristics**:
   - Some algorithms (e.g., Grover) benefit more from optimization
   - Others (e.g., Hamiltonian Simulation) show minimal improvement

3. **Hardware Constraints**:
   - Qubit connectivity significantly impacts SWAP overhead
   - Two-qubit gates are the primary source of errors

### Best Practices

1. Always profile your specific algorithm on target hardware
2. Consider both gate count and circuit depth
3. Higher optimization ≠ always better
4. Use SupermarQ metrics for comprehensive evaluation

---
## Section 8: Extension Ideas & Challenges

### 🚀 Challenge 1: New Algorithm
Implement the Quantum Fourier Transform (QFT) and analyze its transpilation characteristics.

### 🚀 Challenge 2: Custom Backend
Try using a different fake backend (e.g., `Fake27QPulseV1`) and compare results.

### 🚀 Challenge 3: Error Mitigation
Research and implement basic error mitigation techniques (e.g., dynamical decoupling).

### 🚀 Challenge 4: Parameter Sweep
Vary algorithm parameters (e.g., Grover iterations) and study the impact on metrics.

### 🚀 Challenge 5: Real Hardware
If you have IBM Quantum access, run these experiments on real hardware!

In [ ]:
# Workspace for your extensions and challenges

# Your code here...

---
## Conclusion

Congratulations! You've completed the Quantum Transpilation Analysis tutorial. You now understand:

✅ How quantum transpilation optimizes circuits for real hardware  
✅ The trade-offs between different optimization levels  
✅ How to use SupermarQ metrics for circuit evaluation  
✅ How to implement and analyze quantum algorithms systematically  

### Next Steps

1. **Explore** the main.py script for production-ready code
2. **Experiment** with your own quantum algorithms
3. **Read** the referenced papers for deeper theoretical understanding
4. **Contribute** improvements back to the repository

### Resources

- [Qiskit Documentation](https://qiskit.org/documentation/)
- [SupermarQ Paper](https://arxiv.org/abs/2202.11045)
- [Quantum Algorithm Zoo](https://quantumalgorithmzoo.org/)
- [IBM Quantum Experience](https://quantum-computing.ibm.com/)

---

**Happy Quantum Computing! 🎉🔬⚛️**